In [2]:
#Custom gates
#Amitava Chakraborty - 18th Jan 2018
from qiskit import IBMQ
# Loading your IBM Q account(s)
IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [2]:
# Useful additional packages
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from math import pi
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute, transpile
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import state_fidelity
from qiskit import BasicAer

backend = BasicAer.get_backend('unitary_simulator')

In [3]:
#'Barrier' cannot be found in quantum computing textbooks because 
#it is not a standard primitive of quantum information theory like unitary gates and quantum circuits
#The barrier is a directive for circuit compilation 
#to separate pieces of a circuit so that any optimizations or re-writes 
#are constrained to only act between barriers 
#if there are no barriers they act on the whole circuit
#This only comes into play when using the transpile or execute functions in Qiskit 
#(because execute includes a transpile step).

#Example
#If a circuit has several 1-qubit gates in a row acting on the same qubit 
#these can be combined into a single 1-qubit gate. 
#If you wanted to explicitly prevent this sort of behaviour you can place a barrier between them.

#Step 1 : Create a 1-qubit circuit with several gates
qr = QuantumRegister(1)
circuit1 = QuantumCircuit(qr)
circuit1.u1(0.2, qr[0])
circuit1.u2(0.1,0.2, qr[0])
circuit1.u3(0.1, 0.2, 0.3, qr[0])
circuit1.draw()

┌─────────┐┌─────────────┐┌─────────────────┐
q0_0: ┤ U1(0.2) ├┤ U2(0.1,0.2) ├┤ U3(0.1,0.2,0.3) ├
      └─────────┘└─────────────┘└─────────────────┘

In [4]:
#If we transpile it these gates are combined using the default settings
circuit1t = transpile(circuit1)
circuit1t.draw()

┌───────────────────────────┐
q0_0: ┤ U3(1.6629,0.6018,0.43905) ├
      └───────────────────────────┘

In [7]:
original_depth = circuit1.depth()

In [5]:
#Now if we wanted to stop the gates being combined we could add barriers:
qr = QuantumRegister(1)
circuit2 = QuantumCircuit(qr)
circuit2.u1(0.2, qr[0])
circuit2.barrier(qr)
circuit2.u2(0.1,0.2, qr[0])
circuit2.barrier(qr)
circuit2.u3(0.1, 0.2, 0.3, qr[0])
circuit2.draw()

┌─────────┐ ░ ┌─────────────┐ ░ ┌─────────────────┐
q1_0: ┤ U1(0.2) ├─░─┤ U2(0.1,0.2) ├─░─┤ U3(0.1,0.2,0.3) ├
      └─────────┘ ░ └─────────────┘ ░ └─────────────────┘

In [6]:
#In this case transpiling wont change the circuit:
circuit2t = transpile(circuit2)
circuit2t.draw()

┌─────────┐ ░ ┌─────────────┐ ░ ┌─────────────────┐
q1_0: ┤ U1(0.2) ├─░─┤ U2(0.1,0.2) ├─░─┤ U3(0.1,0.2,0.3) ├
      └─────────┘ ░ └─────────────┘ ░ └─────────────────┘

In [8]:
#Barriers do not add to depth
print (original_depth - circuit1.depth())

0
